In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

Способы ввода данных:
1. Ввод с клавиатуры
2. Случайная генерация
3. Импорт файла csv

Перед первым запуском программы требуется установить библиотеки nash и nashpy, для этого запустите пайплайны ниже

In [35]:
pip install nash

Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install nashpy

Note: you may need to restart the kernel to use updated packages.


In [31]:
from prettytable import PrettyTable
from scipy.optimize import linprog
import matplotlib.pyplot as plt
from fractions import Fraction
import random, scipy, csv
import nashpy as nash
import pandas as pd
import numpy as np

In [30]:
way = int(input('Способ ввода данных: '))

# Ввод с клавиатуры
if way == 1:
    sum_strategy_A = int(input('Введите количество стратегий для компании А: '))
    name_rows, name_columns = [], []
    for num in range(sum_strategy_A):
        name = input('Введите название {} стратегии компании А: '.format(num + 1))
        name_rows.append(name)

    sum_strategy_B = int(input('Введите количество стратегий для компании Б: '))
    for num in range(sum_strategy_B):
        name = input('Введите название {} стратегии компании Б: '.format(num + 1))
        name_columns.append(name)
    
    print('\nВведите матрицу весовых коэффициентов для игрока А поэлементно:')
    matrix_A = []
    rows = []
    for num_row, row in enumerate(name_rows, start=1):
        for num_column, column in enumerate(name_columns, start=1):
            value = int(input('Введите элемент строки {} столбца {}: '.format(num_row, num_column)))
            rows.append(value)
        matrix_A.append(rows)
        rows = []
        
    print('\nВведите матрицу весовых коэффициентов для игрока B поэлементно:')
    matrix_B = []
    rows = []
    for num_row, row in enumerate(name_rows, start=1):
        for num_column, column in enumerate(name_columns, start=1):
            value = int(input('Введите элемент строки {} столбца {}: '.format(num_row, num_column)))
            rows.append(value)
        matrix_B.append(rows)
        rows = []

# Случайная генерация
elif way == 2:
    # ----------------------------------
    # Заполняем матрицу для игрока А
    size_rows = int(input("Количество стратегий игрока А: ")) 
    size_cols = size_rows

    # Последовательно генерируем случайные числа типа int в диапазоне (0;1000) и добавляем их в матрицу
    matrix_A = []
    for i in range(size_rows):
        rows = []
        for j in range(size_cols):
            x = random.randint(0,1000)
            rows.append(x)
        matrix_A.append(rows)

    # список для имен строк и столбцов матрицы:
    name_rows = []
    for i in range(size_rows):
        name_rows.append(i+1)
        
    # ----------------------------------
    # Заполняем матрицу для игрока B
    size_rows = int(input("Количество стратегий игрока Б: ")) 
    size_cols = size_rows

    # Последовательно генерируем случайные числа типа int в диапазоне (0;1000) и добавляем их в матрицу
    matrix_B = []
    for i in range(size_rows):
        rows = []
        for j in range(size_cols):
            x = random.randint(0,1000)
            rows.append(x)
        matrix_B.append(rows)

    # список для имен строк и столбцов матрицы:
    name_columns = []
    for i in range(size_cols):
        name_columns.append(i+1)

# Импорт csv файла
elif way == 3:
    # Игрок А
    matrix_A = []
    file_way = input('Введите путь к файлу (матрица игрока А): ')
    with open(file_way, 'r', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=';')
        for row in spamreader:
            matrix_A.append(row)
    
    # Для образования списка с названиями колонок матрицы
    name_rows = matrix_A[0]
    name_rows = name_rows[1:]
    del matrix_A[0]

    # Для образования списка с названиями строк
    name_rows = []
    for num, value in enumerate(matrix_A):
        name_rows.append(value[0])
        del matrix_A[num][0]
        
    # Перевод коэффициентов из str в тип int
    for num_1, row in enumerate(matrix_A):
        for num_2, value in enumerate(row):
            matrix_A[num_1][num_2] = int(value)
    
    # -------------------------------------------------
    # Игрок В
    matrix_B = []
    file_way = input('Введите путь к файлу (матрица игрока Б): ')
    with open(file_way, 'r', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=';')
        for row in spamreader:
            matrix_B.append(row)
    
    # Для образования списка с названиями колонок матрицы
    name_columns = matrix_B[0]
    name_columns = name_columns[1:]
    del matrix_B[0]

    # Для образования списка с названиями строк
    name_columns = []
    for num, value in enumerate(matrix_B):
        name_columns.append(value[0])
        del matrix_B[num][0]
        
    # Перевод коэффициентов из str в тип int
    for num_1, row in enumerate(matrix_B):
        for num_2, value in enumerate(row):
            matrix_B[num_1][num_2] = int(value)


matrix_A_DF = matrix_A
matrix_B_DF = matrix_B

matrix_A = pd.DataFrame(matrix_A)
matrix_A.columns = name_rows
matrix_A.index = name_rows
print('\nВесовая матрица игрока A:\n', matrix_A, '\n')
    
matrix_B = pd.DataFrame(matrix_B)
matrix_B.columns = name_columns
matrix_B.index = name_columns
print('Весовая матрица игрока Б:\n', matrix_B, '\n')


def pure_strategy(matrix_A, matrix_B, name_rows, name_columns, matrix_A_DF, matrix_B_DF):
    min_max_A = list(matrix_A.max())
    min_max_B = list(matrix_B.max(axis=1))
    
    index_column = min_max_A.index(min(min_max_A)) # индекс минимального значения по столбцам в матрице игрока А
    index_row = min_max_B.index(min(min_max_B)) # индекс минимального значения по строкам в матрице игрока В
    
    print('Оптимальная чистая стратегия для игрока A: ', name_rows[index_column])
    print('Цена игры для игрока A при выборе чистой оптимальной стратегии: ', min(min_max_A))
    
    print('Оптимальная чистая стратегия для игрока Б: ', name_columns[index_row])
    print('Цена игры для игрока Б при выборе чистой оптимальной стратегии: ', min(min_max_B))
    
    # -----------------------------
    # Общая (суммарная) цена игры
    print('Общая (суммарная) цена игры: {}'.format(min(min_max_A) + min(min_max_B)))
    
    # -----------------------------
    # Количество равновесий по Нэшу
    nash_A = []
    for num_row, row in enumerate(matrix_A_DF):
        for num_value, value in enumerate(row):
            if value in min_max_A:
                nash_A.append([num_row, num_value])
                
    nash_B = []
    for num_row, row in enumerate(matrix_B_DF):
        for num_value, value in enumerate(row):
            if value in min_max_B:
                nash_B.append([num_row, num_value])
    
    nash = 0
    for blns_A in nash_A:
        for blns_B in nash_B:
            if blns_A == blns_B:
                nash += 1
    print('\nКоличество равновесий по Нэшу: ', nash)          

pure_strategy = pure_strategy(matrix_A, matrix_B, name_rows, name_columns, matrix_A_DF, matrix_B_DF)


def Mix_strategy(matrix_A, matrix_B, name_rows, name_columns):
    matrix_A = np.array(matrix_A)
    matrix_B = np.array(matrix_B)
    
    game = nash.Game(matrix_A, matrix_B)
    equilibria = game.support_enumeration()
    
    mix_strategy_A, mix_strategy_B = [], [] # смешанная стратегия
    help_matrix_A, help_matrix_B = [], []   # вспомогательные матрицы
    price_game_A, price_game_B = 0, 0       # цена игры
    optimal_strategy = None                 # Оптимальная стратегия
    
    
    for strategy in equilibria:
        for row in range(len(matrix_A)):
            for value in range(len(matrix_A[0])):
                help_matrix_A.append(matrix_A[row][value] * strategy[0][row] * strategy[1][value])
                help_matrix_B.append(matrix_B[row][value] * strategy[0][row] * strategy[1][value])
        
        if (sum(help_matrix_A) + sum(help_matrix_B)) >= (price_game_A + price_game_B):
            price_game_A = sum(help_matrix_A)
            price_game_B = sum(help_matrix_B)
            optimal_strategy = strategy
    
    for i in strategy[0]:
        mix_strategy_A.append(str(round(i*100)) + '%')
        
    for i in strategy[1]:
        mix_strategy_B.append(str(round(i*100)) + '%')
        
    
    # Для вывода таблицы PrettyTable
    mytable = PrettyTable()
    mytable.field_names = name_rows     # имена полей таблицы
    mytable.add_row(mix_strategy_A)     # добавление данных по одной строке за раз
    print('\nТаблица смешанных стратегий для компании А:\n' + str(mytable))
    
    print('Цена игры для игрока А при выборе смешанной оптимальной стратегии: {}'.format(round(price_game_A, 3)))
    
    mytable = PrettyTable()
    mytable.field_names = name_columns  # имена полей таблицы
    mytable.add_row(mix_strategy_B)     # добавление данных по одной строке за раз
    print('Таблица смешанных стратегий для компании Б:\n' + str(mytable))
    
    print('Цена игры для игрока Б при выборе смешанной оптимальной стратегии: {}'.format(round(price_game_B, 3)))
    print('Общая цена игры в случае использования оптимальных стратегий: {}'.format(round(price_game_A + price_game_B, 3)))
    

Mix_strategy = Mix_strategy(matrix_A_DF, matrix_B_DF, name_rows, name_columns)

Способ ввода данных: 2
Количество стратегий игрока А: 4
Количество стратегий игрока Б: 4

Весовая матрица игрока A:
      1    2    3    4
1  747  699  634   89
2  115  541  768  672
3   65  760   91  823
4  452  830  891  362 

Весовая матрица игрока Б:
      1    2    3    4
1  841  232  191  466
2  554  832  429  150
3  969  252  415  439
4  348  533  223  276 

Оптимальная чистая стратегия для игрока A:  1
Цена игры для игрока A при выборе чистой оптимальной стратегии:  747
Оптимальная чистая стратегия для игрока Б:  4
Цена игры для игрока Б при выборе чистой оптимальной стратегии:  533
Общая (суммарная) цена игры: 1280

Количество равновесий по Нэшу:  2

Таблица смешанных стратегий для компании А:
+-----+----+----+-----+
|  1  | 2  | 3  |  4  |
+-----+----+----+-----+
| 23% | 0% | 0% | 77% |
+-----+----+----+-----+
Цена игры для игрока А при выборе смешанной оптимальной стратегии: 2290.761
Таблица смешанных стратегий для компании Б:
+-----+-----+----+----+
|  1  |  2  | 3  | 4  |
